In [0]:
%sql
USE CATALOG catproject_catalog

In [0]:
%sql 
USE SCHEMA ecom_silver

In [0]:
%sql
CREATE TABLE ecom_silver.product (
  product_id          STRING,
  product_name        STRING,
  category            STRING,
  brand               STRING,
  description         STRING,
  price               DECIMAL(18,2),
  cost                DECIMAL(18,2),
  currency            STRING,
  status              STRING,
  warehouse_id       STRING,         -- Mã kho (nếu doanh nghiệp có nhiều kho)
  quantity_on_hand   INT,            -- Số lượng thực có trong kho
  quantity_reserved  INT,            -- Đã đặt nhưng chưa xuất
  quantity_available INT,            -- Sẵn sàng để bán = on_hand - reserved
  unit               STRING,         -- Đơn vị tính (e.g., pcs, box)
  created_at          TIMESTAMP,
  updated_at          TIMESTAMP,
  source_system       STRING,
  ingestion_time      TIMESTAMP,
  ingestion_year      INT,
  ingestion_month     INT,
  operation_type      STRING,
  record_hash         STRING,
  source_updated_at   TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month)
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = true,
  'comment' = 'Bảng chứa thông tin sản phẩm từ hệ thống nguồn (ERP, CMS, POS)'
);

In [0]:
%sql
COMMENT ON COLUMN ecom_silver.product.warehouse_id IS 'Kho chứa sản phẩm';
COMMENT ON COLUMN ecom_silver.product.quantity_on_hand IS 'Số lượng có thực tế trong kho';
COMMENT ON COLUMN ecom_silver.product.quantity_reserved IS 'Số lượng đã được đặt giữ chỗ';
COMMENT ON COLUMN ecom_silver.product.quantity_available IS 'Số lượng có thể bán = on_hand - reserved';
COMMENT ON COLUMN ecom_silver.product.unit IS 'Đơn vị tính của sản phẩm';
COMMENT ON COLUMN ecom_silver.product.product_id IS 'Mã sản phẩm duy nhất';
COMMENT ON COLUMN ecom_silver.product.product_name IS 'Tên sản phẩm';
COMMENT ON COLUMN ecom_silver.product.category IS 'Danh mục sản phẩm';
COMMENT ON COLUMN ecom_silver.product.brand IS 'Thương hiệu sản phẩm';
COMMENT ON COLUMN ecom_silver.product.description IS 'Mô tả sản phẩm';
COMMENT ON COLUMN ecom_silver.product.price IS 'Giá bán';
COMMENT ON COLUMN ecom_silver.product.cost IS 'Giá vốn sản phẩm';
COMMENT ON COLUMN ecom_silver.product.currency IS 'Loại tiền tệ';
COMMENT ON COLUMN ecom_silver.product.status IS 'Trạng thái kinh doanh (active/inactive/discontinued)';
COMMENT ON COLUMN ecom_silver.product.created_at IS 'Thời gian tạo bản ghi ở source';
COMMENT ON COLUMN ecom_silver.product.updated_at IS 'Thời gian cập nhật gần nhất ở source';
COMMENT ON COLUMN ecom_silver.product.source_system IS 'Hệ thống nguồn (ERP, CMS, POS)';
COMMENT ON COLUMN ecom_silver.product.ingestion_time IS 'Thời điểm ingest vào lakehouse';
COMMENT ON COLUMN ecom_silver.product.ingestion_year IS 'Năm ingest (partition)';
COMMENT ON COLUMN ecom_silver.product.ingestion_month IS 'Tháng ingest (partition)';
COMMENT ON COLUMN ecom_silver.product.operation_type IS 'INSERT / UPDATE / DELETE';
COMMENT ON COLUMN ecom_silver.product.record_hash IS 'Hash để kiểm tra thay đổi';
COMMENT ON COLUMN ecom_silver.product.source_updated_at IS 'Thời điểm cập nhật dữ liệu tại source';

In [0]:
%sql
ALTER TABLE ecom_silver.product ALTER COLUMN product_id SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN product_name SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN category SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN brand SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN description SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN price SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN cost SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN currency SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN status SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN warehouse_id SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN quantity_on_hand SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN quantity_reserved SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN quantity_available SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN unit SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN created_at SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN updated_at SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN source_system SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN ingestion_time SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN ingestion_year SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN ingestion_month SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN operation_type SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN record_hash SET NOT NULL;
ALTER TABLE ecom_silver.product ALTER COLUMN source_updated_at SET NOT NULL;

ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_status CHECK (status IN ('active', 'inactive', 'discontinued'));
ALTER TABLE ecom_silver.product ADD CONSTRAINT positive_price CHECK (price >= 0);
ALTER TABLE ecom_silver.product ADD CONSTRAINT positive_cost CHECK (cost >= 0);
ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_currency CHECK (currency IN ('USD','VND'));
ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_created_date CHECK (created_at <= current_timestamp());
ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_updated_date CHECK (updated_at <= current_timestamp());
ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_ingestion_date CHECK (ingestion_time <= current_timestamp());
ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_source_updated_date CHECK (source_updated_at <= current_timestamp());
ALTER TABLE ecom_silver.product ADD CONSTRAINT valid_operation_type CHECK (operation_type IN ('INSERT', 'UPDATE', 'DELETE'));

ALTER TABLE ecom_silver.product
ADD CONSTRAINT chk_quantity_on_hand_nonnegative CHECK (quantity_on_hand >= 0);

ALTER TABLE ecom_silver.product
ADD CONSTRAINT chk_quantity_reserved_nonnegative CHECK (quantity_reserved >= 0);

ALTER TABLE ecom_silver.product
ADD CONSTRAINT chk_quantity_available_nonnegative CHECK (quantity_available >= 0);

ALTER TABLE ecom_silver.product
ADD CONSTRAINT chk_ingestion_year_nonnegative CHECK (ingestion_year >= 0);

ALTER TABLE ecom_silver.product
ADD CONSTRAINT chk_ingestion_month_nonnegative CHECK (ingestion_month >= 0);